In [1]:
import re
import pandas as pd
import numpy as np
from plotnine import *
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# nltk.download()

In [2]:
tweets = pd.read_csv('./valorant_tweets.csv')
tweets

,tweet,date,Agent,Role
0,and now i want brimstone in my garden\ni want ...,2021-09-29 22:30:50,Brimstone,Controller
1,why half of the brimstone players are literall...,2021-09-29 22:14:44,Brimstone,Controller
2,@DavidLevy101 @Meloniemaher1 That fucker gets ...,2021-09-29 22:07:50,Brimstone,Controller
3,They also did these cute lil' ghosts! \n\nRobo...,2021-09-29 22:01:21,Brimstone,Controller
4,@AuthorDelaneyW @mis_cue @C_Stroop That is not...,2021-09-29 21:57:18,Brimstone,Controller
...,...,...,...,...
15497,I'm really confused as to why they did what th...,2021-09-29 07:17:43,Kayo,Initiator
15498,@ChidgR @maxblake44 @Foxtel @kayosports @arara...,2021-09-29 07:15:34,Kayo,Initiator
15499,May interested ba \n\nNCT STICKY VER SEALED\n\...,2021-09-29 07:13:49,Kayo,Initiator
15500,@BrokeOnTwice Shout out to all Communities her...,2021-09-29 07:12:59,Kayo,Initiator


In [3]:
tweets.describe()

,tweet,date,Agent,Role
count,15502,15502,15502,15502
unique,15171,13728,16,4
top,"and now i found brimstone in my garden,\ni fou...",2021-09-29 21:45:00,Brimstone,Duelist
freq,13,23,1000,4502


In [4]:
agents = []
loweragents = []
for agent in tweets.loc[:,'Agent'].values:
    if agent not in agents:
        agents.append(agent)
        loweragents.append(agent.lower())
print(agents)

['Brimstone', 'Phoenix', 'Sage', 'Sova', 'Viper', 'Cypher', 'Reyna', 'Killjoy', 'Breach', 'Omen', 'Jett', 'Raze', 'Skye', 'Yoru', 'Astra', 'Kayo']


In [5]:
role = {}
for idx, row in tweets.iterrows():
    if row['Role'] not in role:
        role[row['Role']] = [row['Agent']]
    elif row['Role'] in role and row['Agent'] not in role[row['Role']]:
        role[row['Role']].append(row['Agent'])

In [6]:
tweets

,tweet,date,Agent,Role
0,and now i want brimstone in my garden\ni want ...,2021-09-29 22:30:50,Brimstone,Controller
1,why half of the brimstone players are literall...,2021-09-29 22:14:44,Brimstone,Controller
2,@DavidLevy101 @Meloniemaher1 That fucker gets ...,2021-09-29 22:07:50,Brimstone,Controller
3,They also did these cute lil' ghosts! \n\nRobo...,2021-09-29 22:01:21,Brimstone,Controller
4,@AuthorDelaneyW @mis_cue @C_Stroop That is not...,2021-09-29 21:57:18,Brimstone,Controller
...,...,...,...,...
15497,I'm really confused as to why they did what th...,2021-09-29 07:17:43,Kayo,Initiator
15498,@ChidgR @maxblake44 @Foxtel @kayosports @arara...,2021-09-29 07:15:34,Kayo,Initiator
15499,May interested ba \n\nNCT STICKY VER SEALED\n\...,2021-09-29 07:13:49,Kayo,Initiator
15500,@BrokeOnTwice Shout out to all Communities her...,2021-09-29 07:12:59,Kayo,Initiator


In [20]:
wst = WhitespaceTokenizer()
ps = PorterStemmer()
stop_words = stopwords.words('english')
maxwords = 0
maxindex = 0
with tqdm(total = len(tweets)) as pbar:
    for idx, row in tweets.iterrows():
        pbar.update(1)
        filtered = []
        for word in wst.tokenize(row['tweet']):
            word = word.lower()
            if word not in stop_words and '@' not in word: #remove stopwords and @s
                word = re.sub(r'[^\w]','',word) #remove symbols
                word = re.sub(r'^(http|https)\w+','',word) #remove links
                if word != "":
                    if word not in loweragents:
                        word = ps.stem(word) #get word stem
                    filtered.append(word)
                if(idx == 2274):
                        print(filtered)
            tweets.loc[idx,'words'] = filtered

100%|███████████████████████████████████████████████████████████████████████████| 15502/15502 [00:31<00:00, 487.39it/s]


In [19]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
tweets

,tweet,date,Agent,Role,words
0,and now i want brimstone in my garden\ni want ...,2021-09-29 22:30:50,Brimstone,Controller,"[want, brimstone, garden, want, rose, set, fire]"
1,why half of the brimstone players are literall...,2021-09-29 22:14:44,Brimstone,Controller,"[half, brimstone, player, liter, born, brain]"
2,@DavidLevy101 @Meloniemaher1 That fucker gets ...,2021-09-29 22:07:50,Brimstone,Controller,"[fucker, get, fire, brimstone, version, hell]"
3,They also did these cute lil' ghosts! \n\nRobo...,2021-09-29 22:01:21,Brimstone,Controller,"[also, cute, lil, ghost, robot, ghost, fun, co..."
4,@AuthorDelaneyW @mis_cue @C_Stroop That is not...,2021-09-29 21:57:18,Brimstone,Controller,"[true, fire, brimstone, church, believ, way, c..."
...,...,...,...,...,...
15497,I'm really confused as to why they did what th...,2021-09-29 07:17:43,Kayo,Initiator,"[im, realli, confus, adam, eric, like, gh0rl, ..."
15498,@ChidgR @maxblake44 @Foxtel @kayosports @arara...,2021-09-29 07:15:34,Kayo,Initiator,"[ok, abl, get, kayo, app, smart, tv, use, sams..."
15499,May interested ba \n\nNCT STICKY VER SEALED\n\...,2021-09-29 07:13:49,Kayo,Initiator,"[may, interest, ba, nct, sticki, ver, seal, 55..."
15500,@BrokeOnTwice Shout out to all Communities her...,2021-09-29 07:12:59,Kayo,Initiator,"[shout, commun, like, brother, tgg, fam, well,..."


In [22]:
tweets.iloc[2274]

tweet    @V_Sage_ as you should
date        2021-09-29 22:02:09
Agent                      Sage
Role                   Sentinel
words                        []
Name: 2274, dtype: object

In [26]:
wordfreq = {}
maxwords = 0
maxindex = 0
totalwords = 0
with tqdm(total = len(tweets)) as pbar:
    for idx, row in tweets.iterrows():
        pbar.update(1)
        for word in row['words']:
            if word not in wordfreq.keys():
                wordfreq[word] = 1
            else:
                wordfreq[word]+=1
            totalwords+=1
                
        if len(row['words']) > maxwords:
            maxwords = len(row['words'])
            maxindex = idx
print(wordfreq)

100%|█████████████████████████████████████████████████████████████████████████| 15502/15502 [00:01<00:00, 15479.16it/s]

{'want': 477, 'brimstone': 976, 'garden': 32, 'rose': 38, 'set': 151, 'fire': 497, 'half': 55, 'player': 244, 'liter': 123, 'born': 18, 'brain': 37, 'fucker': 4, 'get': 917, 'version': 64, 'hell': 152, 'also': 336, 'cute': 79, 'lil': 22, 'ghost': 40, 'robot': 7, 'fun': 121, 'concept': 23, 'think': 478, 'omen': 885, 'practic': 42, 'alreadi': 132, 'like': 1141, 'he': 217, 'ghostghost': 3, 'store': 30, 'link': 87, 'kayo': 1017, 'true': 63, 'church': 33, 'believ': 108, 'way': 265, 'christian': 64, 'taught': 7, 'bibl': 23, 'said': 183, 'say': 383, 'it': 684, 'wrong': 77, 'teach': 34, 'lord': 28, 'theyr': 106, 'basic': 54, 'pastorsevangeslist': 1, 'preach': 16, 'via': 154, 'tv': 19, 'usual': 40, 'honkey': 1, 'dori': 1, 'evangel': 13, 'lot': 181, 'scam': 8, 'artistscult': 1, 'leader': 32, 'you': 281, 'die': 84, 'again': 92, 'orvov': 1, 'lostmi': 1, 'bando': 1, 'dhide': 1, 'bodi': 44, 'boot': 8, 'amp': 604, '500k': 1, 'worth': 40, 'item': 37, 'fuck': 238, 'forgot': 25, 'hmm': 10, 'light': 46, 

In [25]:
for i in range(maxwords):
    tweets.loc[:, 'word' + str(i)] = 0
tweets

,tweet,date,Agent,Role,words,word0,word1,word2,word3,word4,...,word37,word38,word39,word40,word41,word42,word43,word44,word45,word46
0,and now i want brimstone in my garden\ni want ...,2021-09-29 22:30:50,Brimstone,Controller,"[want, brimstone, garden, want, rose, set, fire]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,why half of the brimstone players are literall...,2021-09-29 22:14:44,Brimstone,Controller,"[half, brimstone, player, liter, born, brain]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,@DavidLevy101 @Meloniemaher1 That fucker gets ...,2021-09-29 22:07:50,Brimstone,Controller,"[fucker, get, fire, brimstone, version, hell]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,They also did these cute lil' ghosts! \n\nRobo...,2021-09-29 22:01:21,Brimstone,Controller,"[also, cute, lil, ghost, robot, ghost, fun, co...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,@AuthorDelaneyW @mis_cue @C_Stroop That is not...,2021-09-29 21:57:18,Brimstone,Controller,"[true, fire, brimstone, church, believ, way, c...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15497,I'm really confused as to why they did what th...,2021-09-29 07:17:43,Kayo,Initiator,"[im, realli, confus, adam, eric, like, gh0rl, ...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15498,@ChidgR @maxblake44 @Foxtel @kayosports @arara...,2021-09-29 07:15:34,Kayo,Initiator,"[ok, abl, get, kayo, app, smart, tv, use, sams...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15499,May interested ba \n\nNCT STICKY VER SEALED\n\...,2021-09-29 07:13:49,Kayo,Initiator,"[may, interest, ba, nct, sticki, ver, seal, 55...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15500,@BrokeOnTwice Shout out to all Communities her...,2021-09-29 07:12:59,Kayo,Initiator,"[shout, commun, like, brother, tgg, fam, well,...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
with tqdm(total = len(tweets)) as pbar:
    for idx, row in tweets.iterrows():
        pbar.update(1)
        for i in len(row['words']):
            row[word+str(i)] = wordfreq.get(row['words'][i])/totalwords * 

Source: https://www.kaggle.com/mazenramadan/valorant-agents-tweets?select=valorant_tweets.csv

https://www.youtube.com/watch?v=zi16nl82AMA&list=PLQVvvaa0QuDf2JswnfiGkliBInZnIC4HL&index=11
https://www.codespeedy.com/check-if-a-string-contains-a-special-character-or-not-python/
https://stackoverflow.com/questions/4643142/regex-to-test-if-string-begins-with-http-or-https
